# Lin

## Index
1. [Define required clock parameters](#Define-required-clock-parameters)
2. [Download necessary data](#Download-necessary-data)
3. [Load data](#-Load-data)
4. [Extract features and weights](#Extract-features-and-weights)
5. [Load weights into pyaging model](#Load-weights-into-pyaging-model)
6. [Add reference values](#Add-reference-values)
7. [Add preprocessing and postprocesssing steps](#Add-preprocessing-and-postprocesssing-steps)
8. [Check all data objects](#Check-all-data-objects)
9. [Write clock dictionary](#Write-clock-dictionary)
10. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import marshal
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Define required clock parameters

Let's define some required information first:

In [2]:
clock_name = 'lin'
data_type = 'methylation'
model_class = 'LinearModel'
species = 'Homo sapiens'
year = 2016
approved_by_author = '⌛'
citation = "Lin, Qiong, et al. \"DNA methylation levels at individual age-associated CpG sites can be indicative for life expectancy.\" Aging (Albany NY) 8.2 (2016): 394."
doi = "https://doi.org/10.18632/aging.100908"
notes = None

## Download necessary data

In [3]:
# No need, see below

## Load data

In [4]:
# from https://www.aging-us.com/article/100908/text supplement
cpg_sites = [
    "(Intercept)", "cg05228408", "cg16352283", "cg05436231", "cg19046959", 
    "cg17791651", "cg07388493", "cg04036898", "cg07810156", "cg21448423", 
    "cg18660898", "cg25256723", "cg21870884", "cg25947945", "cg09462576", 
    "cg09809672", "cg27553955", "cg27320127", "cg15297650", "cg05331214", 
    "cg24178740", "cg18182399", "cg25431974", "cg24768561", "cg26614073", 
    "cg23320649", "cg12554573", "cg04474832", "cg17421623", "cg22919728", 
    "cg14456683", "cg08209133", "cg16744741", "cg00059225", "cg00489401", 
    "cg02844545", "cg22736354", "cg06493994", "cg03340878", "cg03958979", 
    "cg15804973", "cg13870866", "cg00503840", "cg25762706", "cg25538571", 
    "cg08598221", "cg19724470", "cg07211259", "cg13870494", "cg16386080", 
    "cg00563932", "cg21120249", "cg26581729", "cg17431739", "cg13129046", 
    "cg01560871", "cg06291867", "cg26610808", "cg07621046", "cg13807496", 
    "cg20654468", "cg21992250", "cg15538427", "cg08012287", "cg01820374", 
    "cg19722847", "cg12883767", "cg04123409", "cg22580512", "cg25268718", 
    "cg21296230", "cg21801378", "cg10917602", "cg15195412", "cg20264732", 
    "cg22947000", "cg02228185", "cg01739167", "cg14918082", "cg05379350", 
    "cg08468689", "cg08090640", "cg25809905", "cg05294455", "cg06638433", 
    "cg20366832", "cg19761273", "cg26927807", "cg17471102", "cg02489552", 
    "cg05488632", "cg16363586", "cg17861230", "cg24713204", "cg23679724", 
    "cg03224418", "cg15379633", "cg02994956", "cg23124451", "cg26394940"
]
coefficients = [
    12.2169841, 0.47636173, -5.3124138, 17.7305146, -13.367066, 8.72680959, -4.7759575, 
    10.162153, 15.3892025, -4.4621797, 13.2543665, -11.802998, 22.9981412, -8.2387336, 
    6.3124836, -14.950409, -0.7884001, 48.9368049, -34.306553, 9.83640629, -27.476107, 
    -4.1527608, -1.048605, -4.5917403, -11.443446, 8.70555476, 1.81880164, -26.556597, 
    2.4399993, 0.99214006, 13.1457167, 30.500322, -9.5846721, 36.8195086, 1.98682848, 
    0.38022482, 36.9317174, 66.1611861, 5.95485236, -16.016804, -15.214138, -39.104364, 
    31.2113275, 1.5340163, 10.2956593, 2.62080161, -5.5537073, -12.424324, 19.7417678, 
    -29.24993, -3.5009711, -8.6074197, -7.9914389, 8.22589722, -5.1368284, 13.5034883, 
    13.0769424, -21.374356, 13.6468199, -8.3931276, 8.14605552, 10.5216611, -19.93487, 
    -18.989957, -30.896866, -13.06341, 8.45912249, -10.767354, -0.8383178, 4.06576438, 
    28.1787443, 44.7163476, -6.0293979, 20.050343, -20.618882, -13.217155, -8.6363427, 
    33.8101434, 15.5554908, 17.340667, -16.062905, 8.31318309, -6.0974732, 2.71073045, 
    10.6229217, 2.97899616, -16.331359, 16.5195276, -18.063487, 6.09699424, -11.249025, 
    13.6452671, 17.5027126, -32.487323, 0.25793126, 8.07556639, 15.4139903, -6.4516149, 
    -13.361462, 0.89292205
]

## Extract features and weights

First, let's extract the features and weights:

In [5]:
df = pd.DataFrame({
    'feature': cpg_sites,
    'coefficient': coefficients
})

df.head()

,feature,coefficient
0,(Intercept),12.216984
1,cg05228408,0.476362
2,cg16352283,-5.312414
3,cg05436231,17.730515
4,cg19046959,-13.367066


Then, let's create lists for features and weights. Be careful about the intercept, as it usually shows up as a feature name.

In [6]:
features = df['feature'][1:].tolist()
weights = torch.tensor(df['coefficient'][1:].tolist()).unsqueeze(0)
intercept = torch.tensor([df['coefficient'][0]])

## Load weights into pyaging model

#### Linear model

In [7]:
model = pya.models.LinearModel(input_dim=len(features))

model.linear.weight.data = weights.float()
model.linear.bias.data = intercept.float()

model

LinearModel(
  (linear): Linear(in_features=99, out_features=1, bias=True)
)

## Add reference values

Some clocks have reference values in the case of missing features. It is also possible that these values are for preprocessing features rather than the clock features. Let's add a dictionary with the feature names as the keys.

In [8]:
reference_feature_values = None

## Add preprocessing and postprocesssing steps

The preprocessing and postprocessing objects are dictionaries with the following format, with all items required. It takes in x in the form of a numpy array.

In [9]:
preprocessing = None

Similarly is the case of postprocessing. Remember that your function must be compatible with torch and is applied to each number individually.

In [10]:
postprocessing = None

## Check all data objects

Let's print all data objects to check if they make sense.

#### features

In [11]:
def my_print_function():
    print(f"There are {len(features)} features.")
    print(features)
pya.utils.print_to_scrollable_output(my_print_function)

#### reference_feature_values

In [12]:
def my_print_function():
    if reference_feature_values:
        print(f"There are {len(reference_feature_values)} reference feature values.")
    print(reference_feature_values)
pya.utils.print_to_scrollable_output(my_print_function)

#### preprocessing

In [13]:
def my_print_function():
    print(preprocessing)
    if preprocessing:
        print(preprocessing_helper_objects)
pya.utils.print_to_scrollable_output(my_print_function)

#### postprocessing

In [14]:
def my_print_function():
    print(postprocessing)
    if postprocessing:
        print(postprocessing)
pya.utils.print_to_scrollable_output(my_print_function)

#### weight_dict

In [15]:
def my_print_function():
    for name, param in model.named_parameters():
        print(f"Layer: {name}")
        print(f"Shape: {param.shape}")
        print(param.data)
pya.utils.print_to_scrollable_output(my_print_function)

## Write clock dictionary

Let's put everything together and save:

In [16]:
clock_dict = {
    # Metadata
    'clock_name': clock_name,
    'data_type': data_type,
    'model_class': model_class,
    'species': species,
    'year': year,
    'approved_by_author': approved_by_author,
    'citation': citation,
    'doi': doi,
    "notes": notes,

    # Data
    'reference_feature_values': reference_feature_values if reference_feature_values else None,
    'preprocessing': preprocessing if preprocessing else None, 
    'features': features,
    'weight_dict': model.state_dict(),
    'postprocessing': postprocessing if postprocessing else None,
}

torch.save(clock_dict, f'../weights/{clock_name}.pt')

## Clear directory

Delete all files that are not clock jupyter notebooks:

In [17]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted folder: .ipynb_checkpoints
